In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.signal import ellip, filtfilt

In [2]:
# Загрузка данных
df = pd.read_csv('data.csv', parse_dates=['DATE'], dayfirst=True)

# Преобразуем курс в тип float
df['rate'] = pd.to_numeric(
    df['Euro/Czech koruna (EXR.D.CZK.EUR.SP00.A)'], errors='coerce')

# Удалим строки с пропущенными значениями
df.dropna(inplace=True)

### Этап 1: Удаление выбросов с использованием DBSCAN

In [11]:
dbscan = DBSCAN(eps=0.5, min_samples=5)
df['cluster'] = dbscan.fit_predict(df[['rate']])

# Восстановим выбросы (кластеры с меткой -1) на основе соседних значений
outliers = df['cluster'] == -1
df.loc[outliers, 'rate'] = df['rate'].rolling(window=3, min_periods=1).mean()

### Этап 2: Фильтрация шума
### (a) Статистика Бокса-Пирса для теста на белый шум

In [13]:
ljung_box_result = acorr_ljungbox(df['rate'], lags=[10], return_df=True)
print(f"Статистика Бокса-Пирса:\n{ljung_box_result}")

Статистика Бокса-Пирса:
        lb_stat  lb_pvalue
10  66228.83027        0.0


### (b) Применение эллиптического фильтра

In [14]:
b, a = ellip(5, 0.01, 120, 0.1, btype='low', analog=False)
df['filtered_rate'] = filtfilt(b, a, df['rate'])

### Этап 3: Сглаживание с использованием взвешенного скользящего среднего

In [15]:

df['smoothed_rate'] = df['filtered_rate'].rolling(
    window=5, min_periods=1).mean()

### Этап 4: Проверка стационарности до и после обработки (ADF и KPSS)

In [6]:

# ADF-тест
adf_result_before = adfuller(df['rate'])
adf_result_after = adfuller(df['smoothed_rate'])

# KPSS-тест
kpss_result_before = kpss(df['rate'], regression='c')
kpss_result_after = kpss(df['smoothed_rate'], regression='c')

C:\Users\Sp1r14ual\AppData\Local\Temp\ipykernel_9988\56153983.py:7: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_result_before = kpss(df['rate'], regression='c')
C:\Users\Sp1r14ual\AppData\Local\Temp\ipykernel_9988\56153983.py:8: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_result_after = kpss(df['smoothed_rate'], regression='c')


In [7]:
print(f"ADF-тест до обработки: p-значение = {adf_result_before[1]}")
print(f"ADF-тест после обработки: p-значение = {adf_result_after[1]}")
print(f"KPSS-тест до обработки: p-значение = {kpss_result_before[1]}")
print(f"KPSS-тест после обработки: p-значение = {kpss_result_after[1]}")

ADF-тест до обработки: p-значение = 0.042636539414536115
ADF-тест после обработки: p-значение = 0.021308310239812937
KPSS-тест до обработки: p-значение = 0.01
KPSS-тест после обработки: p-значение = 0.01


In [8]:
# Этап 5: Проверка на наличие тренда методом Фостера-Стьюарта
df['diff'] = df['smoothed_rate'].diff()
trend_foster_stewart = np.sign(df['diff']).sum()

if trend_foster_stewart > 0:
    print("В данных присутствует восходящий тренд.")
elif trend_foster_stewart < 0:
    print("В данных присутствует нисходящий тренд.")
else:
    print("Тренд не выявлен.")

В данных присутствует нисходящий тренд.


In [10]:
# Выводим первые 5 строк для проверки результатов
print(df[['DATE', 'rate', 'filtered_rate', 'smoothed_rate']])

            DATE    rate  filtered_rate  smoothed_rate
0     1999-01-04  35.107      35.101536      35.101536
1     1999-01-05  34.917      35.113236      35.107386
2     1999-01-06  34.850      35.132413      35.115729
3     1999-01-07  34.886      35.163508      35.127673
4     1999-01-08  34.938      35.209378      35.144014
...          ...     ...            ...            ...
6727  2025-01-13  25.230      25.181045      25.166300
6728  2025-01-14  25.294      25.197026      25.173697
6729  2025-01-15  25.218      25.216458      25.185159
6730  2025-01-16  25.231      25.238295      25.200417
6731  2025-01-17  25.265      25.261328      25.218831

[6670 rows x 4 columns]
